# Đại lý Đặt phòng Khách sạn và Vé máy bay Mẫu Ví dụ

Giải pháp này sẽ giúp bạn đặt vé máy bay và khách sạn. Kịch bản là một chuyến đi từ London Heathrow LHR ngày 20 tháng 2 năm 2024 đến New York JFK, trở về ngày 27 tháng 2 năm 2025, bay hạng phổ thông với British Airways thôi. Tôi muốn ở tại khách sạn Hilton ở New York, vui lòng cung cấp chi phí cho vé máy bay và khách sạn.


# Khởi tạo Azure AI Agent Service và lấy thông tin cấu hình từ **.env**

### **.env**

Tạo một file .env

**.env** chứa chuỗi kết nối của Azure AI Agent Service, mô hình được AOAI sử dụng, và dịch vụ API Tìm kiếm Google tương ứng, ENDPOINT, v.v.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Tên Triển Khai Mô Hình Dịch Vụ Azure AI Agent của Bạn"

[**LƯU Ý**] Bạn sẽ cần một mô hình với Giới Hạn Tỷ Lệ 100.000 (Token mỗi phút) và Giới Hạn Tỷ Lệ 600 (Yêu cầu mỗi phút)

  Bạn có thể lấy mô hình trong Microsoft Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Chuỗi Kết Nối Dự Án Dịch Vụ Azure AI Agent của Bạn"

  Bạn có thể lấy chuỗi kết nối dự án trong tổng quan dự án của bạn tại Màn hình Cổng AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Khóa API Tìm Kiếm SERPAPI của Bạn"
- **SERPAPI_SEARCH_ENDPOINT** = "Điểm Cuối Tìm Kiếm SERPAPI của Bạn"

Để lấy Tên Triển Khai Mô Hình và Chuỗi Kết Nối Dự Án của Azure AI Agent Service, bạn cần tạo Azure AI Agent Service. Khuyến nghị sử dụng [mẫu này](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) để tạo trực tiếp （***Lưu ý:*** Azure AI Agent Service hiện được đặt ở một khu vực giới hạn. Khuyến nghị bạn tham khảo [liên kết này](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) để thiết lập khu vực)

Agent cần truy cập SERPAPI. Khuyến nghị đăng ký dùng [liên kết này](https://serpapi.com/searches). Sau khi đăng ký, bạn có thể lấy Khóa API và Điểm Cuối duy nhất.


# Setup 

Để chạy notebook này, bạn cần đảm bảo rằng bạn đã cài đặt các thư viện cần thiết bằng cách chạy `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Phiên bản Semantic Kernel của bạn nên là ít nhất 1.27.2.


Tải cài đặt và tài nguyên từ tệp .env của bạn, vui lòng đảm bảo bạn đã thêm các khóa và cài đặt của mình và tạo một tệp .env cục bộ.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Đăng nhập vào Azure

Bạn bây giờ cần đăng nhập vào Azure. Mở một terminal và chạy lệnh sau:

```bash
az login
```

Lệnh này sẽ yêu cầu bạn nhập thông tin đăng nhập Azure của mình, cho phép dịch vụ Azure AI Agent hoạt động đúng cách.


# Explanation:
Đây là một biến lưu trữ khóa API để truy cập dịch vụ API SERP (Trang Kết Quả Công Cụ Tìm Kiếm). Khóa API là một định danh duy nhất được sử dụng để xác thực các yêu cầu liên quan đến tài khoản của bạn.

Mục đích: Mục đích của dòng này là lưu khóa API trong một biến để có thể sử dụng xác thực các yêu cầu tới dịch vụ API SERP. Khóa API là bắt buộc để truy cập dịch vụ và thực hiện tìm kiếm.
Cách Lấy Khóa API SERP: Để lấy khóa API SERP, thực hiện các bước chung sau tại https://serpapi.com (các bước cụ thể có thể khác nhau tùy thuộc vào dịch vụ API SERP bạn đang sử dụng):

Chọn một Dịch Vụ API SERP: Có nhiều dịch vụ API SERP có sẵn, như SerpAPI, Google Custom Search JSON API, và các dịch vụ khác. Hãy chọn dịch vụ phù hợp nhất với nhu cầu của bạn.

Đăng ký Tài khoản: Truy cập trang web của dịch vụ API SERP bạn chọn và đăng ký một tài khoản. Bạn có thể cần cung cấp một số thông tin cơ bản và xác minh địa chỉ email của mình.

Tạo Khóa API: Sau khi đăng ký, đăng nhập vào tài khoản và điều hướng đến phần API hoặc bảng điều khiển. Tìm tùy chọn để tạo hoặc sinh một khóa API mới.
Sao chép Khóa API vào tệp .env của bạn.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Đây là một biến lưu trữ URL cơ sở cho điểm cuối API SERP. Tên biến BASE_URL là một quy ước được sử dụng để chỉ rằng URL này là điểm khởi đầu để thực hiện các yêu cầu API.
'https://serpapi.com/search':

Đây là chuỗi URL thực tế được gán cho biến BASE_URL. Nó đại diện cho điểm cuối để thực hiện các truy vấn tìm kiếm sử dụng API SERP.

# Purpose:
Mục đích của dòng này là để định nghĩa một hằng số giữ URL cơ sở cho API SERP. URL này sẽ được sử dụng làm điểm khởi đầu để xây dựng các yêu cầu API nhằm thực hiện các hoạt động tìm kiếm.

# Usage:
Bằng cách định nghĩa URL cơ sở trong một biến, bạn có thể dễ dàng tái sử dụng nó trong toàn bộ mã của bạn mỗi khi cần thực hiện yêu cầu đến API SERP. Điều này làm cho mã của bạn dễ bảo trì hơn và giảm nguy cơ lỗi do mã hoá cứng URL ở nhiều nơi. Ví dụ hiện tại là https://serpapi.com/search?engine=bing, sử dụng API tìm kiếm Bing. Các API khác có thể được chọn tại https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Giải thích:

Đây là nơi chứa mã plugin của bạn.

Định nghĩa Lớp: `class BookingPlugin`: Định nghĩa một lớp tên là BookingPlugin chứa các phương thức để đặt khách sạn và vé máy bay.

Phương thức Đặt Khách Sạn:

- `@kernel_function(description="booking hotel")`: Một trình trang trí mô tả hàm như một hàm kernel để đặt khách sạn.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Định nghĩa một phương thức để đặt khách sạn với các tham số được chú thích và kiểu trả về.

Phương thức xây dựng một từ điển các tham số cho yêu cầu đặt khách sạn và gửi một yêu cầu GET đến API SERP. Nó kiểm tra trạng thái phản hồi và trả về thông tin khách sạn nếu thành công, hoặc None nếu yêu cầu thất bại.

Phương thức Đặt Vé Máy Bay:

- `@kernel_function(description="booking flight")`: Một trình trang trí mô tả hàm như một hàm kernel để đặt vé máy bay.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Định nghĩa một phương thức để đặt vé máy bay với các tham số được chú thích và kiểu trả về.

Phương thức xây dựng các từ điển tham số cho yêu cầu chuyến bay đi và chuyến bay về, gửi các yêu cầu GET đến API SERP. Nó kiểm tra trạng thái phản hồi và thêm thông tin chuyến bay vào chuỗi kết quả nếu thành công, hoặc in thông báo lỗi nếu yêu cầu thất bại. Phương thức trả về chuỗi kết quả chứa thông tin chuyến bay.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Giải thích:
Câu lệnh Import: Import các mô-đun cần thiết cho thông tin xác thực Azure, đại lý AI, nội dung tin nhắn chat, vai trò tác giả và trình trang trí hàm kernel.

Trình quản lý ngữ cảnh bất đồng bộ: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Đây thiết lập một trình quản lý ngữ cảnh bất đồng bộ để xử lý thông tin xác thực Azure và tạo một client đại lý AI.

Tên và hướng dẫn đại lý: 
- `AGENT_NAME = "BookingAgent"`: Định nghĩa tên của đại lý.
- `AGENT_INSTRUCTIONS = """..."""`: Cung cấp hướng dẫn chi tiết cho đại lý về cách xử lý các yêu cầu đặt chỗ.

Tạo định nghĩa đại lý: `agent_definition = await client.agents.create_agent(...)`: Tạo định nghĩa đại lý với mô hình, tên và hướng dẫn đã chỉ định.

Tạo đại lý AzureAI: `agent = AzureAIAgent(...)`: Tạo đại lý AzureAI sử dụng client, định nghĩa đại lý và plugin đã định nghĩa.

Tạo Thread: `thread: AzureAIAgentThread | None = None`: Tạo một thread cho đại lý. Không bắt buộc phải tạo thread trước - nếu giá trị `None` được cung cấp, một thread mới sẽ được tạo trong lần gọi đầu tiên và trả về như một phần của phản hồi.

Đầu vào người dùng: `user_inputs = ["..."]`: Định nghĩa danh sách đầu vào của người dùng để đại lý xử lý.

Trong khối finally, xóa thread và đại lý để dọn dẹp tài nguyên.


# Xác thực

Lớp `DefaultAzureCredential` là một phần của Azure SDK cho Python. Nó cung cấp một cách mặc định để xác thực với các dịch vụ Azure. Nó cố gắng xác thực bằng nhiều phương thức theo một thứ tự cụ thể, chẳng hạn như biến môi trường, danh tính được quản lý, và thông tin đăng nhập Azure CLI.

Các Hoạt động Bất đồng bộ: Mô-đun aio chỉ ra rằng lớp DefaultAzureCredential hỗ trợ các hoạt động bất đồng bộ. Điều này có nghĩa là bạn có thể sử dụng nó với asyncio để thực hiện các yêu cầu xác thực không chặn.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tuyên bố từ chối trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi nỗ lực đảm bảo độ chính xác, xin lưu ý rằng bản dịch tự động có thể chứa lỗi hoặc sự không chính xác. Tài liệu gốc bằng ngôn ngữ gốc của nó nên được coi là nguồn chính xác và có thẩm quyền. Đối với những thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp do con người thực hiện. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc giải thích sai nào phát sinh từ việc sử dụng bản dịch này.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
